¿Qué hace este libro?
1. ingresa a la pagina: 'https://www.adres.gov.co/lupa-al-giro/identifica-tu-giro'
2. ingresa en los campos:
*   Fecha Inicio: '01/01/2025'
*   Fecha Fin: '06/10/2025'
*   NIT EPS o IPS Beneficiaria: usa un nit por vez del listado proporcionado

Debes dar tiempo para que cada busqueda se ejecute (entre 2 y 5 seg)

3. Extrae los archivos en formato excel (uno por nit)
4. integra los archivos en csv (ingresando las étiquetas de columnas definidas)

#CONFIGURACIÓN DEL AMBIENTE COLAB

In [5]:
# CHUNK 0: Instalación con Chrome real (no Chromium)
!pip install selenium==4.25.0
!apt-get update
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb
!rm google-chrome-stable_current_amd64.deb

# Verificar
!google-chrome --version
print("✅ Chrome instalado")

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,181 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of 

In [6]:
# CHUNK 1: Imports
import os
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

if not os.path.exists('backups'): os.makedirs('backups')
print("✅ Librerías cargadas")

✅ Librerías cargadas


#MODO ESCRITORIO FORZADO

In [7]:
# CHUNK 2: Configuración del Driver con Chrome real
def configurar_motor_desktop():
    options = Options()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    driver = webdriver.Chrome(options=options)
    return driver

# Test rápido
print("🧪 Probando driver...")
test_driver = configurar_motor_desktop()
test_driver.get("https://www.google.com")
print(f"✅ Driver funciona! Título: {test_driver.title}")
test_driver.quit()

🧪 Probando driver...
✅ Driver funciona! Título: Google


#EXTRACCIÓN Y TRANSFORMACIÓN

In [8]:
# CHUNK 3: Función de captura masiva
def extraer_crudo_nit(driver, nit, f_inicio, f_fin):
    try:
        driver.get('https://www.adres.gov.co/lupa-al-giro/identifica-tu-giro')
        wait = WebDriverWait(driver, 5)

        iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[id*='WebPartWPQ4']")))
        driver.switch_to.frame(iframe)

        driver.execute_script(f"""
            var inputs = document.querySelectorAll('input[type="text"]');
            if(inputs.length >= 3) {{
                inputs[0].value = '{f_inicio}';
                inputs[1].value = '{f_fin}';
                inputs[2].value = '{nit}';
                document.querySelectorAll('input[value="Ver informe"]')[0].click();
            }}
        """)
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        filas_sucias = []

        # Capturamos TODA fila que parezca tener datos (más de 4 celdas)
        for row in soup.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) > 4:
                # Guardamos la fila como una lista de strings pura
                textos = [c.get_text(strip=True) for c in cells]
                # Solo guardamos si hay algo que parezca dinero o el NIT
                if any('$' in t for t in textos) or any(nit in t for t in textos):
                    filas_sucias.append(textos + [nit]) # Guardamos el NIT al final como referencia

        return filas_sucias
    except Exception as e:
        print(f"   ⚠️ Error en NIT {nit}: {str(e)[:50]}")
        return []

#CARGA MASIVA

In [ ]:
# CHUNK 4: Extracción y Limpieza
NITS_PROCESAR = [
    '890303208', '816001182', '802010614', '890307200', '806007650', '807002424',
    '800194798', '890985122', '830011670', '860026123', '892099160', '811000620',
    '812005190', '800095628', '860514592', '800052534', '890941663', '890901475',
    '890501070', '800231604', '817004260', '800165262', '891409291', '800241602',
    '805007737', '890324177', '890901826', '860006560', '891408586', '890500893',
    '801000713', '890102768', '802020334', '17068260', '892300678', '806006237',
    '824000687', '802000608', '890107487', '890208788', '805027911', '24289833',
    '860037950', '811038014', '811045769', '900037353', '900047874', '16703018',
    '900098550', '830023202', '900112351', '900149596', '830110109', '900273686',
    '830512218', '900285194', '900330656', '900116494', '900335780', '900138858',
    '900368444', '900474727', '900236850', '900563107', '830500960', '830020599',
    '890300513', '66917463', '900580962', '900276658', '13487059', '900928616',
    '900699359', '900774610', '901196161', '901002107', '900099945', '890922113',
    '860007336', '860013570', '800149695', '890000381', '828002423', '901429936',
    '900067510', '900352592', '901300333', '900419563', '901212102', '901565478',
    '900073223', '830027158', '901308243', '900989962', '901731685', '891200235',
    '901256347', '900432887', '800130907', '900277244', '830129327', '890331949',
    '900293923', '811028445', '800005727', '900509068', '900413914', '830501223'
]

# Eliminar duplicados manteniendo orden
NITS_PROCESAR = list(dict.fromkeys(NITS_PROCESAR))

EXTRACCION_BRUTA = []

driver = configurar_motor_desktop()

print(f"📡 Iniciando extracción para {len(NITS_PROCESAR)} NITs únicos...")
for i, nit in enumerate(NITS_PROCESAR):
    try:
        print(f"[{i+1}/{len(NITS_PROCESAR)}] 📥 Capturando: {nit}")
        driver.get('https://www.adres.gov.co/lupa-al-giro/identifica-tu-giro')
        wait = WebDriverWait(driver, 15)

        iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[id*='WebPartWPQ4']")))
        driver.switch_to.frame(iframe)

        driver.execute_script(f"""
            var inputs = document.querySelectorAll('input[type="text"]');
            if(inputs.length >= 3) {{
                inputs[0].value = '01/01/2025';
                inputs[1].value = '31/01/2026';
                inputs[2].value = '{nit}';
                document.querySelectorAll('input[value="Ver informe"]')[0].click();
            }}
        """)
        time.sleep(35)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        for row in soup.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) > 5:
                EXTRACCION_BRUTA.append([c.get_text(strip=True) for c in cells])

    except Exception as e:
        print(f"   ❌ Error en NIT {nit}: {str(e)[:50]}")
    driver.switch_to.default_content()

driver.quit()

# --- ETAPA DE LIMPIEZA FINAL ---
print("\n🧹 Iniciando limpieza por anclaje de NIT...")
if EXTRACCION_BRUTA:
    df_raw = pd.DataFrame(EXTRACCION_BRUTA)
    registros_finales = []

    for _, row in df_raw.iterrows():
        cols = [str(c).strip() for c in row if str(c).strip() != ""]

        for idx, cell in enumerate(cols):
            if "NIT-" in cell and (idx + 2) < len(cols):
                try:
                    posible_valor = cols[idx + 2]
                    if '$' not in posible_valor and not any(char.isdigit() for char in posible_valor):
                        continue

                    partes = cell.split('-', 2)
                    if len(partes) >= 3:
                        registros_finales.append({
                            'NIT_IPS': partes[1],
                            'Nombre_IPS': partes[2],
                            'Fecha_Giro': cols[idx + 1],
                            'Valor': posible_valor,
                            'Concepto': cols[idx + 3] if idx + 3 < len(cols) else "N/A",
                            'Entidad': cols[idx + 4] if idx + 4 < len(cols) else "N/A"
                        })
                        break
                except:
                    continue

    df_final = pd.DataFrame(registros_finales)

    if not df_final.empty:
        df_final['Valor'] = (df_final['Valor']
                             .replace(r'[\$,\.]', '', regex=True)
                             .apply(pd.to_numeric, errors='coerce'))

        df_final = df_final.dropna(subset=['Valor'])

        output_name = f"ConsolidadoADRES_{time.strftime('%Y%m%d')}.csv"
        df_final.to_csv(output_name, index=False, encoding='utf-8-sig')
        print(f"🏁 Proceso completado exitosamente.")
        print(f"✅ Registros válidos: {len(df_final)}")
        print(f"📂 Archivo generado: {output_name}")
    else:
        print("❌ No se encontraron datos que coincidan con el patrón de NIT y Valor.")
else:
    print("❌ No se capturaron datos.")

📡 Iniciando extracción para 108 NITs únicos...
[1/108] 📥 Capturando: 890303208
[2/108] 📥 Capturando: 816001182
[3/108] 📥 Capturando: 802010614
[4/108] 📥 Capturando: 890307200
[5/108] 📥 Capturando: 806007650
[6/108] 📥 Capturando: 807002424
[7/108] 📥 Capturando: 800194798
[8/108] 📥 Capturando: 890985122
[9/108] 📥 Capturando: 830011670
[10/108] 📥 Capturando: 860026123
[11/108] 📥 Capturando: 892099160
[12/108] 📥 Capturando: 811000620
[13/108] 📥 Capturando: 812005190
[14/108] 📥 Capturando: 800095628
[15/108] 📥 Capturando: 860514592
[16/108] 📥 Capturando: 800052534
[17/108] 📥 Capturando: 890941663
[18/108] 📥 Capturando: 890901475
[19/108] 📥 Capturando: 890501070
[20/108] 📥 Capturando: 800231604
[21/108] 📥 Capturando: 817004260
[22/108] 📥 Capturando: 800165262
[23/108] 📥 Capturando: 891409291
[24/108] 📥 Capturando: 800241602
[25/108] 📥 Capturando: 805007737
[26/108] 📥 Capturando: 890324177
[27/108] 📥 Capturando: 890901826
[28/108] 📥 Capturando: 860006560
[29/108] 📥 Capturando: 891408586
[30/1